<a href="https://colab.research.google.com/github/ParticleEM/ParEM_neural_latent_variable_model/blob/master/notebooks/neural_latent_variable_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Warning: Work in progress.** Please keep in mind that the following is preliminary. Over the coming weeks, up until the camera-ready date, we will refine the current results, add further results, and clean up the code and presentation, so please check back later. Should the paper be accepted we will include this material in a revision.

# Neural latent variable models for image synthesis and inpainting

In this example, we use PGA to train neural latent variable models for image synthesis and inpainting.


## Dataset description

Our datasets are comprised of $M$ images $y = (y^{m})_{m=1}^M$. We consider two datasets:

- MNIST containing $70,000$ $d_y:=28\times 28$ images of hand-written digits: http://yann.lecun.com/exdb/mnist/. We upscale these samples to $32\times 32$.

- CelebA containing $202,599$ $d_y:=32\times 32$ images of faces of celebrities: https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html.

In both cases, we do not use the entire dataset but a randomly subsampled subset of it ($M=10,000$ and $M=40,000$ for MNIST and CelebA, respectively). In what follows, $M$ denotes the size of this training set. Furthermore, all images' pixel values are normalized so that they lie in $[-1,1]$.

## Model description

The model assumes that each image $y^m$ is generated independently of the others and that it is generated by:

1. drawing a latent variable $x^m$ from a zero-mean unit-variance Gaussian distribution $p(x):=\mathcal{N}(x|0,I_{d_x})$ on a low-dimensional latent space ($\mathbb{R}^{d_x}$ with $d_x$ being $10$ and $64$ for MNIST and CelebA respectively);
2. mapping $x^m$ to the image space $\mathbb{R}^{d_y}$ via a decoder $f_\theta$: a neural network parameterized by some parameters $\theta$ in $\mathbb{R}^{D_\theta}$;
3. adding zero-mean $\sigma^2$-variance Gaussian noise: $y^m=f_\theta(x^m)+\epsilon^m$ where $(\epsilon^m)_{m=1}^M$ is a sequence of i.i.d. R.V.s with law $\mathcal{N}(0,\sigma^2 I_{d_y})$.

In full, the model's density is given by
$$
p_\theta (x,y) = \prod_{m=1}^M p_\theta(x^{m}, y^{m}),\tag{1}$$
where
$$
p_\theta(x^m,y^m)= p_\theta(y^m|x^m)p(x^m),\quad\textrm{with}\quad p_\theta(y^m|x^m) := \mathcal{N}(y^m|f_\theta(x^m), \sigma^2 I_{d_y}).
$$

For $f_\theta$ we use a convolutional neural network with an architecture emulating that used in [[1](https://link.springer.com/chapter/10.1007/978-3-030-58539-6_22)], see below for details. In total, it has $300,161$ and $357,507$ parameters for MNIST and CelebA respectively. 

Note that the model has $d_x$ latent variables per training image, coming out to a total of $D_x=M\times d_x$ variables. That is, $100,000$ in the case of MNIST and $2,560,000$ in that of CelebA.

### Network architecture

The neural networks has is composed of layers of $4$ basic types:

*   $l_\theta$: fully-connected linear layers,
*   $c_\theta$: convolutional layers,
*   $c_\theta^T$: transpose convolutional layers,
*   $b_\theta:$ batch normalization layers.

These are interweaved with GELU activation functions $\phi$. In particular, they are assembled to create $2$ further types of layers:

*   'projection' layers $\pi_\theta:=\phi \circ b_\theta\circ c_\theta \circ \phi\circ b_\theta\circ l_\theta$;
*   'deterministic' layers $d_\theta=\phi \circ b_\theta \circ c_\theta \circ \phi\circ b_\theta \circ c_\theta + I$ where $I$ denotes the identity operator (i.e., the layer has a skip connection).

The network itself then consists of a projection layer followed by two deterministic layers, a transpose convolutional layer, and a $\tanh$ activation:

$$f_\theta =  \tanh\circ c_\theta^T\circ d_\theta \circ d_\theta \circ \pi_\theta.$$

Please the code in [model.py](https://github.com/ParticleEM/ParEM_neural_latent_variable_model/blob/master/parem/model.py) for more details. Or, alternatively, see [[1](https://link.springer.com/chapter/10.1007/978-3-030-58539-6_22), Table 4] for layer-specific input, intermediate, and output dimensions.

## Model training

Training the model entails searching for parameters $\theta_*$ maximizing the marginal likelihood $\theta\mapsto p_\theta(y):=\int p_\theta(x,y)dx$, at least locally. To do so, we use PGA slightly modified to better cope with the high evaluation cost of the log-likelihood's, $\ell(\theta,x):=\log(p_\theta(x,y))$'s, gradients. In particular, in the $\theta$-update we replace $\nabla_{\theta} \ell(\theta,x)$ unbiased estimator thereof obtained by subsampling the training set:

\begin{align*}\nabla_{\theta} \ell(\theta,x)&=\sum_{m=1}^M \nabla_\theta\log(p_\theta(x^m,y^m))=M\left[\frac{1}{M}\sum_{m=1}^M \nabla_\theta\log(p_\theta(x^m,y^m))\right]\\
&\approx M\left[\frac{1}{M_{\mathcal{B}}}\sum_{m\in\mathcal{B}}\nabla_\theta\log(p_\theta(x^m,y^m))\right]=\frac{M}{M_{\mathcal{B}}}\sum_{m\in\mathcal{B}}\nabla_\theta\log(p_\theta(x^m,y^m)),\end{align*}

where $\mathcal{B}$ denotes a random subset of $[M]:=\{1,\dots, M\}$ and $M_\mathcal{B}$ its cardinality. To mitigate the varying magnitudes among $\nabla_\theta\log(p_\theta(x^m,y^m))$'s entries and improve the learning, we use a modified version of the 'heuristic fix' discussed in Section 2.1 of manuscript: we rescale each entry by a scalar, the only difference being that, this time, we allow the scalars to vary with the iteartion count $k$. We choose these scalars as in RMSprop \[[2](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)\]. In full, we update the parameter estimates $\theta_k$ using

$$\tag{2}
    \theta_{k+1} = \theta_k + h\frac{\ M}{NM_{\mathcal{B}}}\lambda\Lambda_k\left[\sum_{n=1}^N \sum_{m\in\mathcal{\cal{B}_{k+1}}}\nabla_\theta\log p_{\theta_k}
(X^{n,m}_k, y^{m}) \right],
$$

where $(X^n)_{n=1}^N=((X^{n,m})_{m=1}^M)_{n=1}^N$ denotes the particle cloud at the $k^{th}$ iteration,  $\Lambda_k$ a diagonal matrix containing the RMSprop step sizes, $\lambda$ a scalar that we tune by hand to mitigate differences between the scales of log-likelihood's $\theta$ and $x$ gradients, and $\mathcal{B}_k$ indexes the image batch used in the $k^{th}$ parameter update.

Because the dimensionality $d_x$ of the latent variables is at least $5000$ times smaller than that of the parameters, the cost of the particle updates is $5000$ smaller than that of the $\theta$ updates (without subsampling) and, so, we are not forced to subsample the training set in the particle updates. Moreover, subsampling in parameter updates but not in particle updates is a common strategy in the literature (e.g., [[3](https://dl.acm.org/doi/abs/10.1145/1390156.1390290), Section 4.4]). Hence, we update the particles without any subsampling just as in standard PGA. Given the product-form structure in (1), these updates read 

\begin{align*}
X^{n,m}_{k+1}&=X^{n,m}_k + h\nabla_x \log p_{\theta_k}
(X^{n,m}_k, y^{m}) + \sqrt{2h} W^{n,m}_k \quad \forall m\in[M],\, n\in [N].\tag{3}
\end{align*}

In total, we run the updates (2,3) $K$ times.

## Implementation details and code

The code necessary to implement the above, and to reproduce the results below, can be found in the following two notebooks:

- [MNIST](https://github.com/ParticleEM/ParEM_neural_latent_variable_model/blob/master/notebooks/MNIST.ipynb). Here, we used $M=10000$, $d_x=10$, $N=10$, $M_{\mathcal{B}}=128$, $h=10^{-4}$, $\lambda=10^{-3}$, $\sigma^2=0.3^2$, and $K=2765$. 
- [CelebA](https://github.com/ParticleEM/ParEM_neural_latent_variable_model/blob/master/notebooks/CelebA.ipynb). Here, we used $M=40000$, $d_x=64$, $N=10$, $M_{\mathcal{B}}=128$, $h=10^{-4}$, $\lambda=2.5\times 10^{-4}$, $\sigma^2=0.3^2$, and $K=3756$.

## Image synthesis

To evaluate the learned decoder $f_\theta$, we examine the fidelity of images generated with it. For reasons related to the persistence of the chains used during training (cf. the discussion at the end of this cell), we do not generate images by sampling latent variables from the prior $p(dx)$ and mapping them through $f_\theta$. Instead, we fit a Gaussian distribution to the empirical distribution of the latent variables featuring in our final particle cloud:

$$\frac{1}{NM}\sum_{n=1}^N\sum_{m=1}^M\delta_{X^{n,m}_K}(dx)\approx\mathcal{N}(dx|\mu,\Sigma),$$

where $\mu,\Sigma$ denote the empirical mean and covariance of the latent variables in the cloud,

$$\mu:=\frac{1}{NM}\sum_{n=1}^N\sum_{m=1}^MX^{n,m}_K,\quad \Sigma:=\frac{1}{NM-1}\sum_{n=1}^N\sum_{m=1}^M(X_K^{n,m}-\mu)(X_K^{n,m}-\mu)^T.$$

We then synthesize images by drawing samples from $\cal{N}(\mu,\Sigma)$ and mapping them through $f_\theta$. Following this approach, we obtained the following images.

### MNIST
![MNIST](https://drive.google.com/uc?export=view&id=1fufmS4RAI0wLx4Lm2xwRbXJ49m3P6zLZ)

### CelebA

![CelebA](https://drive.google.com/uc?export=view&id=1q_ntTlTB8EekYEKiIQ5XooSGiZ4DY0gM)


**Why sample $\mathcal{N}(\mu,\Sigma)$ rather than $p$?** The manner in which the parameters are updated in (2) implies that the decoder is trained to generate high fidelity images only in areas of the latent space that the particles spend significant amounts of time (i.e., areas that our posterior approximations assign significant mass to). In our experiments, the latent variables rapidly moved away from the prior's center of mass and remained in other areas of the latent space. To generate samples in these areas, we sample from $\mathcal{N}(\mu,\Sigma)$ instead of the prior $p$. In our experiments, this lead to marked improvement in the quality of the synthesized images.

## Inpainting

Here, we use the trained decoder $f_\theta$ to recover images $y=(y_i)_{i=1}^{d_y}$ that have been corrupted by masking some of their pixels. To do so, we follow the approach taken in [[1](https://link.springer.com/chapter/10.1007/978-3-030-58539-6_22), Section 5.3]: we search for latent variables that maximize the likelihood of the corrupted image $y_c$,

$$
x_{mle} = \textrm{argmin}_{x\in\mathbb{R}^{dx}} \log p (y_{c}|x)= \textrm{argmin}_{x\in\mathbb{R}^{dx}} \|y_c - f_\theta(x)\|^2_2=\textrm{argmin}_{x\in\mathbb{R}^{dx}}\sum_{i\not\in\mathcal{M}}[y_{i}-f_\theta(x)_i]^2,\tag{4}
$$

where $\mathcal{M}$ indexes the masked pixels. Then, we recover the image by mapping $x_{mle}$ through the decoder.

Below, we show such inpaintings for both datasets. In each row, the leftmost plot displays the corrupted image $y_c$, the middle plot displays the recovered image $f_\theta(x_{mle})$, and the rightmost plot displays the original image $y$. To optimize in (4) we ran Adam [[4](https://arxiv.org/abs/1412.6980)] for $1000$ steps using a step size of $0.01$.


### MNIST

![MNIST](https://drive.google.com/uc?export=view&id=1ZCPEn01cRmbRrwOsGnRlwgRWATRXgQPZ)

![MNIST](https://drive.google.com/uc?export=view&id=1g1SdDk9J4JNNfCI9ojUhCgVpN2P5tXB8)

![MNIST](https://drive.google.com/uc?export=view&id=1g8zEzAXanvhIFjeRGFEYcowsEX9Izb98)

### CelebA

![CELEB](https://drive.google.com/uc?export=view&id=1rz9ms0Y_v0FD_iAX_1p_EAMml7GcIAww)

![CELEB](https://drive.google.com/uc?export=view&id=1sL4APECis1DKBmYjFDNocw5SUKG3LCY-)

![CELEB](https://drive.google.com/uc?export=view&id=1s-HWe_rm1AVAufzEmXQi3Sp4OlWJNJfB)

## References

- [[1](https://link.springer.com/chapter/10.1007/978-3-030-58539-6_22)] E. Nijkamp, B. Pang, T. Han, L. Zhou, S.-C. Zhu & Y. N. Wu, "Learning Multi-layer Latent Variable Model via Variational Optimization of Short Run MCMC for Approximate Inference". European Conference on Computer Vision 2020, p. 361--378.
- [[2](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)] Lecture 6e of Geoffrey Hinton's Coursera Course. 
- [[3](https://www.cs.toronto.edu/~tijmen/pcd/pcd.pdf)] T. Tieleman, "Training Restricted Boltzmann Machines using Approximations to the Likelihood Gradient". ICML 2008, p. 1064--1071.
- [[4](https://arxiv.org/abs/1412.6980)] D. P. Kingma & J. L. Ba, "Adam: a Method for Stochastic Optimization". ICLR 2015. 